In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib    
import pandas as pd
import scipy as sc
import csv
import math
import random
from sklearn.model_selection import train_test_split
from scipy.sparse import isspmatrix, csc_matrix, csr_matrix
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import svd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import io

In [2]:
from skopt import gp_minimize
from skopt.space import Real, Integer

In [3]:
eps                  = np.finfo(float).eps

In [4]:
def createList(r1,r2):
    return [item for item in range(r1,r2 + 1)]

In [5]:
def userSim(Y): 
    print("In userSim")
    N,M              = Y.shape  
    
    Y01              = Y.copy()
    Y01[Y01>0]       = 1
    
    tmp              = (Y**2)@(Y01.T)
    tmp              = tmp + tmp.T
    tmp              = tmp - 2*(Y@(Y.T))
    
    simScore         = 1.0/(1 + np.sqrt(tmp))
    
    nominator        = Y01@Y01.T
    
    denominator      = np.sum(Y01,axis=1).reshape(-1,1)
    denominator      = np.matlib.repmat((denominator),1,N)
    denominator      = denominator + denominator.T
    denominator      = denominator - nominator
    
    B1               = nominator/denominator
    B1[np.isnan(B1)] = 0
    
    simScore         = simScore*B1
    return simScore

In [6]:
def evecs(A,nEvecs):
    npix             = A.shape[0]
    useSparse        = isspmatrix(A)
    
    dd               = 1/((np.sum(A,axis=0))+eps)
    dd               = np.sqrt(dd)

    if(useSparse):
        DD           = sc.sparse.diags(dd)
    else:
        DD           = np.diag(dd)
    
    L                = DD@A@DD
    
    if(useSparse==1):
        ss,v         = eigsh(L,nEvecs,sigma=1)
        ss           = np.flip(ss)
        V            = np.flip(v,axis=1)
    else:
        u,ss,v       = svd(L)
        V            = u[:, :nEvecs]
        
    ss               = ss.reshape(-1,1)
    ss               = ss[:nEvecs]
    
    return V,ss

In [7]:
def spectralClu(A=None,K=None): 
    # A: Affinity Matrix, Higher value -> more similar
    # K: Number of CLuster
    n                = A.shape[0]
    
    D                = np.sum(A,axis=1).reshape(-1,1)
    D[D==0]          = eps
    D                = sc.sparse.spdiags((1.0/np.sqrt(D)).T,0,n,n)
    
    L                = D@A@D
    V,evals          = evecs(L,K)
    
    Vnorm            = (np.linalg.norm(V,axis=1) + eps).reshape(-1,1)
    
    V                = V/Vnorm

    kmeans           = KMeans(n_clusters=K,max_iter=maxiter,random_state=0).fit(V)
    idx              = kmeans.predict(V)

    return idx

In [8]:
def aggregratePrediction(X,UGidx):
    n,m                  = X.shape
    K1                   = len(np.unique(UGidx))
    ctmp_d               = np.ones(n)
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))

    GFreq                = np.bincount(UGidx.reshape(-1,)).reshape(-1,1)
    XPerA                = (ctmp@X)/np.matlib.repmat((GFreq),1,m)
    
    
    return XPerA

In [9]:
def MAE(a,b):
    return np.sum(abs(a*(b!=0) - b))/np.sum(b!=0)

In [10]:
def RMSE(X,Y):
    tmp                        = ((Y - X)*(Y!=0))**2
    
    return math.sqrt(np.sum(tmp)/np.sum(Y!=0))

In [11]:
def AUC(Ypre,Yt,cutoff):
    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    auc                        = 0
    uWithPair                  = 0
    for user in range(N):
        userRating             = Yt[user,:]
        userPrediction         = Ypre[user, :]
        ratedIdx               = userRating>0
        binUserRating          = 2*(userRating[ratedIdx]>=cutoff)-1
        predForKnown           = userPrediction[ratedIdx]
        
        posIdx                 = np.where(binUserRating==1)
        posIdx                 = posIdx[0].T

        negIdx                 = np.where(binUserRating==-1)
        negIdx                 = negIdx[0]
        
        if(np.size(posIdx)!=0 and np.size(negIdx)!=0):
            negIdxGrid         = np.matlib.repmat(negIdx,len(posIdx),1)
            ttlNeg             = np.size(negIdxGrid)
            negIdxGrid         = negIdxGrid.reshape(ttlNeg,1,order='F')
            posIdxGrid         = np.matlib.repmat(posIdx.reshape(-1,1),len(negIdx),1)
            pairs              = np.concatenate((negIdxGrid,posIdxGrid),axis=1)
            pairsPred          = np.concatenate((pairs,predForKnown[negIdxGrid],predForKnown[posIdxGrid]),axis = 1)
            pairsPredWithScore = np.concatenate([pairsPred,(0.5*(pairsPred[:,2]==pairsPred[:,3]).reshape(-1,1)),(1*(pairsPred[:,2]<pairsPred[:,3]).reshape(-1,1))],axis=1)
            auc                = auc + (np.sum(pairsPredWithScore[:,4:6]))/pairsPredWithScore.shape[0]
            uWithPair          = uWithPair + 1    
    auc                        = auc/uWithPair
    
    return auc

In [12]:
def precAtK(Ypre,Yt,k,cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    prec                       = np.zeros((1,k))

    for user in range(N):
        userRating             = Yt[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        userPrediction         = Ypre[user, :]
        prediction             = userPrediction[ratedIdx]
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(min(kNo+1,nz),eps)
            prec[:,kNo]        = prec[:,kNo] + len(intrsct)/mx
            
    prec                       = prec/N
    
    return prec

In [13]:
def recallAtK(Ypre,Yt, k, cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: recall@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    recall                     = np.zeros((1,k))
    
    for user in range(N):
        userRating             = Yt[user, :]
        userPrediction         = Ypre[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        prediction             = userPrediction[ratedIdx]
        
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(len(ratedRelevantIdx),eps)
            recall[:,kNo]      = recall[:,kNo] + len(intrsct)/mx
            
    recall                     = recall/N
    
    return recall 

In [14]:
def ndcgAtk(Ypre,Yt,k):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    res                        = np.zeros((1,k))
    cnt                        = 0
    N,M                        = Yt.shape

    for user in range(N):
        ratedIdx               = Yt[user,:]!=0

        userRating             = Yt[user,ratedIdx]
        userPrediction         = Ypre[user,ratedIdx]
        nz                     = len(userRating)

        ranks                  = np.zeros((1,nz))
        ideal_ranks            = np.zeros((1,nz))
        
        I                      = np.argsort(-userPrediction,axis=0)
        ideal_I                = np.argsort(-userRating,axis=0)

        ranks                  = I
        ideal_ranks            = ideal_I

        oriOrder               = np.argsort(ideal_ranks,axis=0)

        nominator              = userRating/(np.log(ranks+2))
        denominator            = userRating/(np.log(ideal_ranks + 2))
    
        nominator              = nominator[oriOrder]
        denominator            = denominator[oriOrder]
        
        if k > nz:
            nominator          = np.concatenate((nominator, np.zeros((k - nz))))
            denominator        = np.concatenate((denominator, np.zeros((k - nz))))
        elif k < nz:
            nominator          = nominator[0:k]
            denominator        = denominator[0:k]          

        if np.array(np.where(np.cumsum(denominator)== 0)).shape[1] != 0:
            tmp                = np.zeros((1,k))
        else:
            tmp                = np.cumsum(nominator) / np.cumsum(denominator)
            cnt                = cnt + 1
        
        res                    = res + tmp
        
    res                        = res/cnt
    
    return res

In [15]:
def EvaluationAllUpdated(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)
    auc                        = AUC(Yprd, Y1, cutoff)
    precision                  = precAtK(Yprd,Y1, k, cutoff)
    recall                     = recallAtK(Yprd,Y1, k, cutoff)
    f1                         = (2*precision*recall)/(precision + recall + eps)
    ndcg                       = ndcgAtk(Yprd,Y1,k)

    return mae,rmse,auc,precision,recall,f1,ndcg

In [16]:
def EvaluationAllUpdated_N(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)

    return mae,rmse

In [17]:
cols                     = ["Model","MAE","RMSE","AUC","Precision@1","Precision@5","Precision@10","Precision@20",
                            "Precision@30","Precision@40","Recall@1","Recall@5","Recall@10","Recall@20","Recall@30",
                            "Recall@40","F1@1","F1@5","F1@10","F1@20","F1@30","F1@40","NDCG@1","NDCG@5","NDCG@10",
                            "NDCG@20","NDCG@30","NDCG@40"]
Result_Trn               = pd.DataFrame(columns=cols)

In [18]:
cols                     = ["Model","MAE","RMSE"]
Result_Trn_CDR               = pd.DataFrame(columns=cols)
Result_Tst_CDR = pd.DataFrame(columns=cols)

In [19]:
def gradUnifiedLS(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx): 
    
    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    UG                   = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
    VG                   = v[n*d+m*d+K1*d:].reshape(K2,d,order='F')

    UGmatMU              = (UG[UGidx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGidx.reshape(-1,),:]) - V
    
    X                    = U@(V.T)
    Ygt0                 = Y>0
    YMXgt0               = (Y - X)*Ygt0

    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))

    lossobj              = 0
    lossobj              = lossobj + (0.5*(np.sum(YMXgt0**2)))
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2))
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    
    dU                   = dU - YMXgt0@V
    dV                   = dV - (YMXgt0.T)@U

    dU                   = dU - (lambda1*UGmatMU)
    dV                   = dV - (lambda1*VGmatMV)

    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = lambda1*((ctmp)@UGmatMU)
    
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGidx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = lambda1*((ctmp)@VGmatMV)
    
    obj                  = regobj + lossobj
   
    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dUG.flatten('F'),dVG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [20]:
def gradUnifiedMMMF(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGIdx,VGIdx):

    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    theta                = v[n*d+m*d:n*d+m*d+n*(l-1)].reshape(n,l-1,order='F')
    UG                   = v[n*d+m*d+n*(l-1):n*d+m*d+n*(l-1)+K1*d].reshape((K1,d), order='F')
    VG                   = v[n*d+m*d+n*(l-1)+K1*d:n*d+m*d+n*(l-1)+K1*d+K2*d].reshape((K2,d), order='F')
    thetaG               = v[n*d+m*d+n*(l-1)+K1*d+K2*d:].reshape((K1,l-1), order='F')
        
    UGmatMU              = (UG[UGIdx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGIdx.reshape(-1,),:]) - V
    thetaGMtheta         = (thetaG[UGIdx.reshape(-1,),:]) - theta
    
    X                    = U@V.T
    Ygt0                 = Y>0
    BX                   = X*Ygt0
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    dtheta               = np.zeros((n, l-1))
    
    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))
    
    lossobj              = 0
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2) + np.sum(thetaGMtheta**2))
    
    for k in range(l-1):
        S                = Ygt0 - 2*(Y>k+1)
        BZ               = (theta[:,k].reshape(-1,1)@(np.ones([1,m])))*S - BX*S
        lossobj          = lossobj + sum(sum(h(BZ)))
        tmp              = hprime(BZ)*S
        dU               = dU - tmp@V
        dV               = dV - tmp.T@U
        dtheta[:,k]      = tmp@np.ones((m,))
        
    dU                   = dU - lambda1*UGmatMU
    dV                   = dV - lambda1*VGmatMV
    dtheta               = dtheta - lambda1*thetaGMtheta
    
    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGIdx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = np.multiply(lambda1,(ctmp@UGmatMU))
    dthetaG              = np.multiply(lambda1,(ctmp@thetaGMtheta))
 
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGIdx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = np.multiply(lambda1,(ctmp@VGmatMV))
    
    obj                  = regobj + lossobj;            # obj is the objective function that we need to minimize

    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dtheta.flatten('F'),dUG.flatten('F'),dVG.flatten('F'),dthetaG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [21]:
def conjgrad(x0,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx):
    J                     = []
    temp                  = 0
    nu                    = 0.1
    abstol                = 0                                 # stop if gradient magnitude goes below this
    allowNonDecrease      = 0                                 # don't stop if line search fails to find decrease
    digits                = 12                                # digits of precision to use for objective comparisons
    ogfun                 = objGrad
    ogcalls               = 0
    x                     = x0
    numiter               = 0
    j                     = 0
    alpha                 = 10**(-10)
    ogcalls               = ogcalls + 1
    obj,dx,lossobj,regobj = ogfun(x,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    r                     = -dx
    s                     = r
    dirn                  = s
    deltanew              = (r.T)@dirn
    deltazero             = deltanew
    
    while ((numiter<maxiter) and (abs(deltanew)>tol*tol*abs(deltazero)) and (abs(deltanew)>abstol)):
        numiter           = numiter + 1
        j                 = j + 1
        print('\n %.4f' %obj)
        J.append(obj)
        prevobj           = obj

        if (alpha < 10**(-10)):
            alpha         = 10**(-10)

        alpha,obj,dx,ogc  = cgLineSearch(x,obj,dx,dirn,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lossobj,regobj)
        ogcalls           = ogcalls + ogc
        
        temp              = temp + alpha
        x                 = x + alpha*dirn
        r                 = -dx
        deltaold          = deltanew
        deltamid          = r.T@s
        deltanew          = r.T@r
        beta              = (deltanew - deltamid) / deltaold
        dirn              = r + max(0, beta)*dirn
        if ((deltamid/deltanew >= nu) or (dirn.T@dx >= 0)):
            dirn          = r
            j             = 0
        s                 = r
        print(numiter)
        
    return x,numiter,ogcalls,J

In [22]:
def cgLineSearch(x0,obj0,dx0,direction,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lobj,robj):
    seciter                   = 5                               # maximum number of quadratic interpolation iterations
    alpha0                    = alpha
    c1                        = 10**(-4)                        # required decrease in objective (relative to gradient)
    digits                    = 12                              # digits of precision to use for objective comparisons
    gamma                     = 10
    ogfun                     = objGrad
    
    if (alpha0 <= 0):                                           # check initial values
        print('alpha0 must be greater than zero')
    
    obj                       = obj0                            # begin line search
    dx                        = dx0
    etazero                   = (dx.T)@direction
    etaprev                   = etazero
    alpha                     = alpha0
    ogcalls                   = 0
    lossobj                   = lobj
    regobj                    = robj
    
    alpha,obj,dx,lossobj,regobj,ogcalls = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    
    obj,dx,lossobj,regobj     = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    ogcalls                   = ogcalls + 1
    
    oldalpha,oldobj,olddx                           = saveAlpha(alpha,obj,dx)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)

    beta                      = alpha
    eta                       = dx.T@direction
    i                         = 0
    
    while((abs(eta)>c2*abs(etazero)) and (i<seciter) and (pround(obj,digits) <= pround(obj0,digits)) and (etaprev!=eta) and (np.sum((x0 + alpha*direction) != x0)>0)):
        beta                  = eta*beta / (etaprev - eta)
        oldalpha,oldobj,olddx = saveAlpha(alpha,obj,dx)
        alpha                 = alpha + beta
        if(alpha<=0):
            alpha             = 1
        etaprev               = eta
        i                     = i + 1
        obj,dx,lossobj,regobj = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        eta                   = dx.T@direction

    alpha,obj,dx,lossobj,regobj,ogcalls             = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)
    checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction)
    
    return alpha,obj,dx,ogcalls 

In [23]:
def findNonZeroAlpha(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj):
    if(np.array_equal((x0+alpha*direction),x0)):                # Make sure alpha isn't smaller than level of precision
        preAlpha              = alpha
        
        while(np.array_equal((x0+alpha*direction),x0)):
            alpha             = alpha*gamma
        
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        
    return alpha,obj,dx,lossobj,regobj,ogcalls

In [24]:
def backtrack(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx):
    #ensures either (1) Armijo lowex0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcallsr objective, or (2) infinitesimal alpha
    doBacktrack               = 0
    preAlpha                  = alpha
    while((pround(obj,digits)>pround(obj0 + c1*alpha*etazero, digits)) and (np.sum((x0+alpha*direction)!=x0)>0)):
        if(ogcalls > 1):
            if((oldalpha > alpha/gamma) and (oldobj < (obj0+c1*oldalpha*etazero))):
                alpha,obj,dx  = restoreAlpha(oldalpha,oldobj,olddx)
                doBacktrack   = 1
                break
        alpha                 = alpha/gamma
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        doBacktrack           = 1
    return alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack

In [25]:
def saveAlpha(alpha,obj,dx):
    #make sure current step yields decrease in objective
    oldalpha   = alpha
    oldobj     = obj
    olddx      = dx
    return oldalpha,oldobj,olddx

In [26]:
def restoreAlpha(oldalpha,oldobj,olddx):
    alpha      = oldalpha
    obj        = oldobj
    dx         = olddx
    return alpha,obj,dx

In [27]:
def pround(x,d):
    d          = round(d)
    if (d<1):
        print('Number of digits must be integer d = %.4e \n'%d)
    if(x==0 or math.isnan(x) or math.isinf(x)):
        y      = x   
    else:
        p      = math.floor(math.log10(abs(x)))+1
        factor = 10**(d-p)
        y      = np.round(x*factor)/factor
    return y

In [28]:
def checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction):
    if round(obj, digits) >= round(obj0, digits):
        print('Warning: Finished line search without decreasing objective.\nMay have reached limit of precision, or obj/grad code may be broken.\n')
    if etazero == eta:
        print('Warning: Line search yielded no change in directional derivative.\nMay have reached limit of precision.\n')
    if sum(x0 + alpha * direction != x0) == 0:
        print('Warning: Line search yielded no change in position.\nMay have reached limit of precision.\n')

In [29]:
def h(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01/2 - zin01*z + zin01*(z**2)/2 + zle0/2 - zle0*z
    return ret  

In [30]:
def hprime(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01*z - zin01 - zle0
    return ret

In [31]:
def m3fSoftmax(xy,theta):
    n,m        = xy.shape
    n1,l1      = theta.shape
    if(n!=n1):
        print('sizes of xy and theta don''t match');
    y          = np.ones((n,m))
    for i in range(l1):
        tmp    = ((theta[:,i]).reshape(-1,1))@(np.ones((1,m)))
        tmp    = xy>=tmp
        y      = y + tmp
    return y

## Loading Matrices for Video Games domain

In [32]:
from pandas import read_csv

In [33]:
rating_matrix_video = np.loadtxt("rating_matrix_video.csv", delimiter=",")

In [34]:
temp_d = read_csv('new_user_ids_video_3k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_video_3k = temp_np.reshape(temp_np.shape[0],)

In [35]:
temp_d = read_csv('item_ids_video_3k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_video_3k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Loading Matrics for Movies and TV domain

In [197]:
rating_matrix_mt = np.loadtxt("rating_matrix_mt.csv", delimiter=",")

In [198]:
temp_d = read_csv('new_user_ids_mt_3k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_mt_3k = temp_np.reshape(temp_np.shape[0],)

In [199]:
temp_d = read_csv('item_ids_mt_3k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_mt_3k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Removing overlapping test user_ids from target domain

In [200]:
d                        = 100 

In [201]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_3k, new_user_ids_mt_3k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 ... 'A12HTLSN3JJNZ0' 'A12HXKVHVSVPC3' 'A12HXKY2F5J3U3']
(3000,)


In [202]:
train = np.random.choice(overlapping_user_ids_t, size=math.ceil(0.8*overlapping_user_ids_t.shape[0]), replace=False)
test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

In [203]:
X_train = np.empty((0,100))
Y_train = np.empty((0,100))
X_test = np.empty((0,100))
Y_test = []

In [204]:
for user_id in test:
    ind1 = np.where(new_user_ids_mt_3k == user_id)
    Y_test.append(rating_matrix_mt[ind1][0])
    new_user_ids_mt_3k =np.delete(new_user_ids_mt_3k, ind1, 0)
    rating_matrix_mt = np.delete(rating_matrix_mt, ind1, 0)

In [205]:
Y_test = np.array(Y_test)
print(Y_test.shape)

(600, 12686)


In [206]:
print(new_user_ids_mt_3k.shape)
print(rating_matrix_mt.shape)

(6400,)
(6400, 12686)


## GRS for Video Games domain

In [46]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                              #Number of Clusters in User Space
K2                       = 22                              #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 1
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [47]:
n, m = rating_matrix_video.shape
print(n, m)

9000 8238


In [48]:
#Spectral Clustering
simU                     = userSim(rating_matrix_video)
simU                     = (simU + simU.T)/2
UGidx_video              = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_video.T)
simI                     = (simI + simI.T)/2
VGidx_video              = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


In [49]:
print(UGidx_video)
print(UGidx_video.shape)

[[3]
 [3]
 [3]
 ...
 [0]
 [2]
 [2]]
(9000, 1)


In [50]:
print(VGidx_video)
print(VGidx_video.shape)

[[ 0]
 [ 7]
 [ 0]
 ...
 [ 1]
 [12]
 [19]]
(8238, 1)


In [51]:
unique, frequency = np.unique(UGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [5670  403  320  361  398  348  411  327  384  378]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
Frequency Values: [555 540 428 310 389 464 473 254 522 141 296 204 556 151 408 445 277 454
 395 394 183 399]


### Recommendation

In [52]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [53]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_video,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_video,VGidx_video)
U_video                   = v[0:n*d].reshape(n,d,order='F')
V_video                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_video                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_video@V_video.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_video)
XPerAMat            = XPerA[UGidx_video.reshape(-1,),:]

XPer_Grp                    = UG_video@V_video.T
XPer_GrpMat                 = XPer_Grp[UGidx_video.reshape(-1,),:]
XPer_GrpMat[XPer_GrpMat<=1] = 1
XPer_GrpMat[XPer_GrpMat>=5] = 5


 3523170.7967
1

 3069094.7163
2

 1877994.9241
3

 1682344.8025
4

 1671940.9563
5

 1625892.3385
6

 1547242.9691
7

 1498416.7745
8

 951284.3315
9

 718006.2269
10

 535280.3586
11

 389576.6651
12

 310046.1300
13

 266110.1853
14

 239968.0732
15

 226410.6133
16

 215549.5064
17

 209569.3524
18

 197820.1002
19

 187096.9984
20

 183496.9800
21

 173215.0946
22

 163277.4510
23

 155566.6775
24

 144251.8597
25

 136109.7440
26

 133330.5003
27

 127628.8785
28

 126048.2622
29

 123165.0415
30

 119392.9341
31

 116996.4443
32

 110868.8625
33

 108826.6899
34

 103417.8052
35

 101080.1237
36

 96394.0223
37

 94528.3042
38

 91666.2168
39

 89959.1010
40

 87748.1029
41

 85838.7021
42

 84280.2155
43

 82729.5287
44

 81748.1615
45

 80264.3443
46

 79524.4664
47

 78245.5865
48

 77604.6237
49

 76139.6125
50

 75392.5708
51

 74224.7458
52

 73736.8016
53

 73068.7793
54

 72606.7781
55

 71979.9882
56

 71341.3193
57

 70752.0864
58

 70013.0990
59

 69446.9095
60

 686

In [54]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat, rating_matrix_video, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Video Games domain - Run 1",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

## GRS for Movies and TV domain

In [207]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                               #Number of Clusters in User Space
K2                       = 100                             #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 30
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [208]:
n, m = rating_matrix_mt.shape
print(n, m)

6400 12686


In [209]:
#Spectral Clustering
simU                     = userSim(rating_matrix_mt)
simU                     = (simU + simU.T)/2

UGidx_mt                    = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_mt.T)
simI                     = (simI + simI.T)/2

VGidx_mt                    = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


C:\Users\NIT\AppData\Local\Temp\ipykernel_16492\3829436230.py:21: RuntimeWarning: invalid value encountered in true_divide
  B1               = nominator/denominator


In [210]:
print(UGidx_mt)
print(UGidx_mt.shape)

[[5]
 [1]
 [1]
 ...
 [1]
 [6]
 [2]]
(6400, 1)


In [211]:
print(VGidx_mt)
print(VGidx_mt.shape)

[[90]
 [75]
 [ 5]
 ...
 [54]
 [13]
 [60]]
(12686, 1)


In [212]:
unique, frequency = np.unique(UGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [ 544 2653  421  524  326  446  299  413  389  385]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Frequency Values: [305 205   4  14  18 428 325  14 182  36  25 147  13 114  47 146 186 293
 233  20 189  28 255 237 112  61   3 196 183  12 254 321 159 258  10 191
 249   4   6  17   7 156  92 280  77   3   4 205 220 326  15 118 111 296
 327  12   7  18 177 318 266   5  19 143 158  18  10   1  31   3 232   4
 213   7   6 217 206   3 141 107   3 256 278   8 193   5 304   8 296 306
 178   4 320  91   3  83 199   4  82   6]


### Recommendation

In [215]:
np.random.seed(1)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [216]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_mt,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_mt,VGidx_mt)
U_mt                  = v[0:n*d].reshape(n,d,order='F')
V_mt                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_mt                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_mt@V_mt.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_mt)
XPerAMat_mt            = XPerA[UGidx_mt.reshape(-1,),:]

XPer_Grp                    = UG_mt@V_mt.T
XPer_GrpMat_mt                 = XPer_Grp[UGidx_mt.reshape(-1,),:]
XPer_GrpMat_mt[XPer_GrpMat_mt<=1] = 1
XPer_GrpMat_mt[XPer_GrpMat_mt>=5] = 5


 59685361.0235
1

 49530560.4814
2

 33634731.9286
3

 31136289.7804
4

 30074603.7628
5

 28974867.9571
6

 27008525.0556
7

 24695832.4980
8

 23872826.4670
9

 20944060.8182
10

 17956481.6722
11

 15604464.8099
12

 14807899.1134
13

 11514873.1980
14

 8559275.6424
15

 8105089.9913
16

 6386827.9806
17

 4998687.5138
18

 4457379.8298
19

 3946204.8465
20

 3242928.4891
21

 2784496.2490
22

 2639036.5811
23

 2360430.1285
24

 2110265.2406
25

 2058135.3476
26

 1818516.1144
27

 1678321.3572
28

 1617163.5753
29

 1469963.2381
30

 1290114.7225
31

 1231060.3905
32

 1184168.3544
33

 1080410.1786
34

 968702.2929
35

 907174.8834
36

 877385.5781
37

 792022.2530
38

 710368.6059
39

 632996.1149
40

 613005.1266
41

 532920.8625
42

 470805.3031
43

 458960.5813
44

 427252.0222
45

 378940.8082
46

 346184.5742
47

 338482.9431
48

 311903.3604
49

 292150.5389
50

 283644.1982
51

 274182.4453
52

 257427.7985
53

 246528.1424
54

 244435.9734
55

 238633.6548
56

 229361.

In [217]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat_mt, rating_matrix_mt, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_mt",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Movies & TV domain - Run 3",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [218]:
Result_Trn

,Model,MAE,RMSE,AUC,Precision@1,Precision@5,Precision@10,Precision@20,Precision@30,Precision@40,...,F1@10,F1@20,F1@30,F1@40,NDCG@1,NDCG@5,NDCG@10,NDCG@20,NDCG@30,NDCG@40
0,Video Games domain - Run 1,1.349378,1.555223,0.810751,0.829556,0.813498,0.812091,0.811322,0.811155,0.811104,...,0.824260,0.825509,0.825728,0.825782,0.941221,0.989732,0.994650,0.996355,0.996710,0.996808
1,Movies & TV domain - Run 1,1.207732,1.445411,0.767572,0.903438,0.883018,0.880687,0.879508,0.879230,0.879016,...,0.888443,0.892977,0.894551,0.895183,0.907331,0.975776,0.987275,0.992740,0.994441,0.995365
2,Movies & TV domain - Run 2,1.211265,1.390602,0.816999,0.901094,0.880773,0.878447,0.877119,0.876654,0.876417,...,0.886738,0.891000,0.892291,0.892858,0.904459,0.975703,0.987042,0.992485,0.994090,0.994795
3,Movies & TV domain - Run 3,1.310733,1.493622,0.813545,0.900469,0.880523,0.878228,0.877072,0.876669,0.876456,...,0.886542,0.890953,0.892308,0.892889,0.903523,0.975740,0.987127,0.992403,0.994233,0.994970


In [196]:
# Result_Trn.drop(index=Result_Trn.index[-1],axis=0,inplace=True)

## Creating X, Y matrices for mapping

In [219]:
#Adding individual latent vectors
for user_id in train:
    source_ind = np.where(new_user_ids_video_3k == user_id)
    target_ind = np.where(new_user_ids_mt_3k == user_id)
    X_train = np.concatenate((X_train, U_video[source_ind]), axis = 0)
    Y_train = np.concatenate((Y_train, U_mt[target_ind]), axis = 0)
    
for user_id in test:
    source_ind = np.where(new_user_ids_video_3k == user_id)
    target_ind = np.where(new_user_ids_mt_3k == user_id)
    X_test = np.concatenate((X_test, U_video[source_ind]), axis = 0)  

    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2400, 100)
(2400, 100)
(600, 100)
(600, 12686)


In [220]:
groupsToMembers_video = []
groupsToMembers_mt = []
for i in range(K1):
    groupsToMembers_video.append([])
    groupsToMembers_mt.append([])

for i in range(UGidx_video.shape[0]):
    groupsToMembers_video[int(UGidx_video[i][0])].append(new_user_ids_video_3k[i])
    
for i in range(UGidx_mt.shape[0]):
    groupsToMembers_mt[int(UGidx_mt[i][0])].append(new_user_ids_mt_3k[i])
    
print(len(groupsToMembers_video))
print(len(groupsToMembers_mt))

10
10


In [221]:
for i in range(len(groupsToMembers_video)):
    print(len(groupsToMembers_video[i]))

5670
403
320
361
398
348
411
327
384
378


In [222]:
for i in range(len(groupsToMembers_mt)):
    print(len(groupsToMembers_mt[i]))

544
2653
421
524
326
446
299
413
389
385


In [223]:
print(groupsToMembers_mt[5])

['A10WAEKV9HS5FI', 'A1PW71P0CLM96Z', 'ACH3KKYZ8KQD4', 'AE2I3FDKJGJH7', 'A10Q1S3RAYZISW', 'A1L1QPV4VH7ZWQ', 'A3U2G9PS31CJJ', 'A106A4W1IPIBEP', 'A10ZOXRR0QMZ9Z', 'A1PN8DHG6OTUKF', 'A5PH6RF27N547', 'A1R6WGR0PZU2F3', 'A2XY528MUE532M', 'A3VFJ9CZRLR3HB', 'A2ZGEWI84EQ8PD', 'A10VY8IWC7E3CN', 'A15QJN1R15P73I', 'A13K0GI233WFNH', 'AA4I1D8DAQ0FG', 'A11WHRS0C94AK6', 'A2I45MTJY7588C', 'A2YZCN03UX54BA', 'A2UE51GVP0OIRG', 'A1TQXEL5LMYZKU', 'A29PVSXRXX8HJG', 'ADL9SVF4FHJPL', 'A10WSZ7BPAABIA', 'A268PL29SC28LR', 'A04665232QBWBWC3DQOXI', 'A11YUCEJ399PHL', 'A1SZLUCH8EKLCO', 'A2XUJR3FY50DDY', 'A1CVPHNN7XI8ZA', 'A3LXWFIWK8X7P2', 'A2ZUDSDATN5KKK', 'A1E1BTLNFUKG5', 'A2BSF99EM2B3A2', 'A2BIOXEYGR2SQU', 'A1IZ3HYALXW83T', 'A1EZTWYL70LSAE', 'A3P2IVHCDNOB82', 'A11OAK8NM6O8VO', 'A9HHZB7DZ505Y', 'A10J4L673JCW9X', 'A11DYOHW4EF63Q', 'A12BG5M4Y4006J', 'A128BDVEHE06O3', 'A127H8IPNWV271', 'APUA4KZ6IEPXA', 'A2KCXZ9U2YJQJF', 'A1CXTY9RRTF7MK', 'A1ZEC13OIHJIIX', 'A1MOK1YQ6V2HPD', 'A30Z3P5LRHEHXH', 'A1MS4DV6SAAZS2', 'A11PGOUZIY

In [224]:
counter = 0
for i in range(len(groupsToMembers_video)):
    currGroup_video = groupsToMembers_video[i]
    for j in range(len(groupsToMembers_mt)):
        currGroup_mt = groupsToMembers_mt[j]
        if len(set(currGroup_video).intersection(set(currGroup_mt))) > 0:
            counter = counter + 1
            X_train = np.row_stack((X_train, UG_video[i]))
            Y_train = np.row_stack((Y_train, UG_mt[j]))
            
print(X_train.shape)
print(Y_train.shape)
print(counter)

(2500, 100)
(2500, 100)
100


## MLP Mapping

In [225]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [226]:
class GetDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [227]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(100, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 100)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [228]:
# Set fixed random number seed
torch.manual_seed(43)

In [229]:
dataset = GetDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=X_train.shape[0], shuffle=False, num_workers=0)

In [174]:
# space = [
#     Real(1e-3, 5e-2, name='lr'),
#     Real(1e-5, 1e-2, name='decay')
# ]

In [175]:
# def trainMLPAndGetMAE(hyperparameters):
#     # Extract hyperparameters
#     learning_rate = hyperparameters[0]
#     wt_decay = hyperparameters[1]
    
#     mlp = MLP()
  
#     # Define the loss function and optimizer
#     loss_function = nn.MSELoss()
#     optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate, weight_decay=wt_decay)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)
    
#     counter = 0
#     # Run the training loop
#     while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum

#         # Print epoch
#     #     print(f'Starting epoch {epoch+1}')

#         # Set current loss value
#         current_loss = 0.0
#         losses = []
#         # Iterate over the DataLoader for training data
#         for i, data in enumerate(trainloader, 0):

#           # Get and prepare inputs
#     #         if counter > 2:
#     #             break
#     #         print(counter)
#             inputs, targets = data
#             inputs, targets = inputs.float(), targets.float()
#             targets = targets.reshape((targets.shape[0], 100))

#     #         print("Inputs shape: ", inputs.shape)
#     #         print("Target shape: ", targets.shape)
#     #         counter += 1

#           # Zero the gradients
#             optimizer.zero_grad()

#           # Perform forward pass
#             outputs = mlp(inputs)
#     #         print("Output shape: ", outputs.shape)

# #             outputs_np = outputs.detach().numpy()
# #             targets_np = targets.detach().numpy()
#             t_V_mt = torch.from_numpy(V_mt.T).float()

# #             t_vec = targets_np@V_mt.T
# #             o_vec = outputs_np@V_mt.T
#             t_vec = torch.matmul(targets, t_V_mt)
#             o_vec = torch.matmul(outputs, t_V_mt)
#     #         print("t_vec.shape: ", t_vec.shape)
#     #         print("o_vec.shape: ", o_vec.shape)
#             indicator_matrix = (t_vec >= 0.7)
#     #         print("indicator matrix: ", indicator_matrix)
#     #         print(np.all(indicator_matrix == True))
# #             t_vec_interacted = torch.from_numpy(t_vec*indicator_matrix).requires_grad_() 
# #             o_vec_interacted = torch.from_numpy(o_vec*indicator_matrix).requires_grad_() 
#             t_vec_interacted = torch.mul(t_vec, indicator_matrix)
#             o_vec_interacted = torch.mul(o_vec, indicator_matrix)
# #             t = torch.from_numpy(a)

#           # Compute loss
#             loss = loss_function(o_vec_interacted, t_vec_interacted)
#     #         print("Loss: ", loss.item())
#             print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
#             losses.append(loss.item())
#           # Perform backward pass
#             loss.backward()

#           # Perform optimization
#             optimizer.step()


#         mean_loss = sum(losses) / len(losses)
#         scheduler.step(mean_loss)
# #         print(f"Loss at epoch {counter + 1} = {mean_loss}")
#         if optimizer.param_groups[-1]['lr'] <= 1e-4:
#             break;
#         counter += 1

#     # Process is complete.
#     print('Training process has finished.')
    
#     pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
#     print("pred_X_train: ", pred_X_train.shape)
#     par1 = pred_X_train@V_mt.T
#     par2 = Y_train@V_mt.T
#     mae = MAE(par1, par2)
#     print("MAE: ", mae)
#     print()
#     print()
#     return mae

In [176]:
# def objective(hyperparameters):
#     print(hyperparameters)
#     return trainMLPAndGetMAE(hyperparameters)

In [177]:
# Perform Bayesian optimization
# result = gp_minimize(objective, space, n_calls=20)

In [178]:
# # Get the best hyperparameters and performance
# # best_hyperparameters = {
# #     'K1' = result.x[0]
# #     'K2' = result.x[1]
# #     'lambda1UnifiedLS' = result.x[2]
# #     'lambda3UnifiedLS' = result.x[3]
# # }
# best_performance = result.fun
# print("Best MAE: {:.4f}".format(result.fun))
# print("Best Hyperparameters: {}".format(dict(zip(['lr', 'decay'], result.x))))

In [179]:
# # Print the MAE values and hyperparameters for each call
# for i, val in enumerate(result.func_vals):
#     print("Call {}: MAE={:.4f}, Hyperparameters={}".format(i, val, result.x_iters[i]))

In [180]:
# # Print the values of the optimized hyperparameters
# print('Optimized Hyperparameters:')
# for hyperparameter, value in zip(space, result.x):
#     print('{}: {}'.format(hyperparameter.name, value))

## MLP with tuned hyperparameters

In [230]:
torch.manual_seed(43)

In [231]:
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.018113971388685, weight_decay=0.0052151663029590236)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)

In [232]:
counter = 0
# Run the training loop
while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum
    
    # Print epoch
#     print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    losses = []
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
#         if counter > 2:
#             break
#         print(counter)
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 100))
        
#         print("Inputs shape: ", inputs.shape)
#         print("Target shape: ", targets.shape)
#         counter += 1
      
      # Zero the gradients
        optimizer.zero_grad()
      
      # Perform forward pass
        outputs = mlp(inputs)
#         print("Output shape: ", outputs.shape)
        
#         outputs_np = outputs.detach().numpy()
#         targets_np = targets.detach().numpy()
        t_V_mt = torch.from_numpy(V_mt.T).float()
    
#         t_vec = targets_np@V_mt.T
#         o_vec = outputs_np@V_mt.T
#         print("t_vec.shape: ", t_vec.shape)
#         print("o_vec.shape: ", o_vec.shape)
        t_vec = torch.matmul(targets, t_V_mt)
        o_vec = torch.matmul(outputs, t_V_mt)
        indicator_matrix = (t_vec >= 0.7)
#         print("indicator matrix: ", indicator_matrix)
#         print(np.all(indicator_matrix == True))
#         t_vec_interacted = t_vec*indicator_matrix
#         o_vec_interacted = o_vec*indicator_matrix
        t_vec_interacted = torch.mul(t_vec, indicator_matrix)
        o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

        # Compute loss
        loss = loss_function(o_vec_interacted, t_vec_interacted)
        
#         print("Loss: ", loss.item())
        print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
        losses.append(loss.item())
      # Perform backward pass
        loss.backward()
      
      # Perform optimization
        optimizer.step()
        
        
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    print(f"Loss at epoch {counter + 1} = {mean_loss}")
    if optimizer.param_groups[-1]['lr'] <= 1e-4:
        break;
    counter += 1

# Process is complete.
print('Training process has finished.')

Epoch : 1 | Error : 11.255547523498535 | LR : 0.018113971388685
Loss at epoch 1 = 11.255547523498535
Epoch : 2 | Error : 4.5829620361328125 | LR : 0.018113971388685
Loss at epoch 2 = 4.5829620361328125
Epoch : 3 | Error : 2.414663553237915 | LR : 0.018113971388685
Loss at epoch 3 = 2.414663553237915
Epoch : 4 | Error : 0.7111101150512695 | LR : 0.018113971388685
Loss at epoch 4 = 0.7111101150512695
Epoch : 5 | Error : 0.7501546740531921 | LR : 0.018113971388685
Loss at epoch 5 = 0.7501546740531921
Epoch : 6 | Error : 0.9471696615219116 | LR : 0.018113971388685
Epoch 00006: reducing learning rate of group 0 to 1.8114e-03.
Loss at epoch 6 = 0.9471696615219116
Epoch : 7 | Error : 0.5805624127388 | LR : 0.0018113971388685002
Loss at epoch 7 = 0.5805624127388
Epoch : 8 | Error : 0.5371806025505066 | LR : 0.0018113971388685002
Loss at epoch 8 = 0.5371806025505066
Epoch : 9 | Error : 0.4912480413913727 | LR : 0.0018113971388685002
Loss at epoch 9 = 0.4912480413913727
Epoch : 10 | Error : 0.45

In [233]:
pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
print(pred_X_train.shape)
cutoff = 3
k = 40
par1 = pred_X_train@V_mt.T
par2 = Y_train@V_mt.T
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Training Metrics")
# print(mae, rmse)

(2500, 100)


In [234]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Training_Metrics                = ["CDGRS_Train",mae,rmse]
Result_Trn_CDR.loc[len(Result_Trn_CDR)]       = ["Train-3",mae,rmse]

In [235]:
pred_X_test = mlp(torch.from_numpy(np.float32(X_test))).detach().numpy()
print(pred_X_test.shape)
cutoff = 3
k = 40
par1 = pred_X_test@V_mt.T
par1[par1 > 5] = 5
par2 = Y_test
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Test Metrics")
# print(mae, rmse)

(600, 100)


In [236]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Testing_Metrics                = ["CDGRS_Test",mae,rmse]
Result_Tst_CDR.loc[len(Result_Tst_CDR)]       = ["Test-3",mae,rmse]

In [237]:
Result_Trn_CDR

,Model,MAE,RMSE
0,Train-1,1.030749,1.203379
1,Train-2,0.529898,0.662577
2,Train-3,0.542004,0.670203


In [238]:
Result_Trn_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_16492\3129751611.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Trn_CDR.mean()


MAE     0.700884
RMSE    0.845387
dtype: float64

In [239]:
Result_Tst_CDR

,Model,MAE,RMSE
0,Test-1,1.748930,1.928951
1,Test-2,1.046947,1.307687
2,Test-3,1.069449,1.314708


In [240]:
Result_Tst_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_16492\661790523.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Tst_CDR.mean()


MAE     1.288442
RMSE    1.517115
dtype: float64

In [192]:
# Result_Trn_CDR.drop(index=Result_Trn_CDR.index[-1],axis=0,inplace=True)
# Result_Tst_CDR.drop(index=Result_Tst_CDR.index[-1],axis=0,inplace=True)